In [1]:
import pandas as pd
import numpy as np
import datetime
import datetime as dt
import pytz
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import statsmodels.api as sm  
import matplotlib.pyplot as plt  
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/louisaliparisa/Desktop


In [3]:
def dateparse(time_in_secs):
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))

path = working_directory + '/data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv'
bitcoin1 = pd.read_csv(path, parse_dates=[0], date_parser=dateparse)

In [4]:
bitcoin1.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00+00:00,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,2011-12-31 02:53:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31 02:54:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31 02:55:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31 02:56:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
bitcoin = bitcoin1.dropna().reset_index(drop=True)
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00+00:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 10:50:00+00:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,2011-12-31 11:59:00+00:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,2011-12-31 12:00:00+00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,2011-12-31 23:16:00+00:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3613764,2021-03-30 19:56:00+00:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
3613765,2021-03-30 19:57:00+00:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
3613766,2021-03-30 19:58:00+00:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
3613767,2021-03-30 19:59:00+00:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [7]:
bitcoin['Timestamp'] = bitcoin['Timestamp'].dt.tz_localize(None)
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 10:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,2011-12-31 11:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,2011-12-31 12:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,2011-12-31 23:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3613764,2021-03-30 19:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
3613765,2021-03-30 19:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
3613766,2021-03-30 19:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
3613767,2021-03-30 19:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [8]:
bitcoin = bitcoin.groupby([pd.Grouper(key='Timestamp', freq='D')]).first().reset_index()
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2012-01-01,4.84,4.84,4.84,4.84,10.000000,48.400000,4.840000
2,2012-01-02,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
3,2012-01-03,5.32,5.32,5.32,5.32,2.419173,12.870000,5.320000
4,2012-01-04,4.93,4.93,4.93,4.93,9.680000,47.722400,4.930000
...,...,...,...,...,...,...,...,...
3373,2021-03-26,52449.23,52537.81,52449.23,52527.35,2.742287,143985.370640,52505.587896
3374,2021-03-27,54821.66,54856.01,54821.66,54856.01,0.261643,14346.416748,54832.004555
3375,2021-03-28,56095.81,56158.59,56095.81,56149.01,0.898928,50459.981106,56133.517533
3376,2021-03-29,55361.25,55417.15,55361.25,55401.67,1.314104,72804.822995,55402.653850


In [9]:
bitcoin = bitcoin.set_index('Timestamp')

In [10]:
bitcoinx = bitcoin[['Close']]
bitcoinx

,Close
Timestamp,
2011-12-31,4.39
2012-01-01,4.84
2012-01-02,5.00
2012-01-03,5.32
2012-01-04,4.93
...,...
2021-03-26,52527.35
2021-03-27,54856.01
2021-03-28,56149.01


In [11]:
bitcoinx['Close'] = bitcoinx['Close'].fillna(method='ffill')

<ipython-input-11-8db6d492bfdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoinx['Close'] = bitcoinx['Close'].fillna(method='ffill')


In [13]:
bitcoinx.index = pd.to_datetime(bitcoinx.index)

In [14]:
#print(bitcoin.head(20))


splitdate = '2018-12-17'
predate = '2020-04-01'
bitcoin_train = bitcoinx.loc[bitcoinx.index > splitdate]
bitcoin_train = bitcoin_train.loc[bitcoin_train.index <= predate]
bitcoin_test = bitcoinx.loc[bitcoinx.index > predate]
#bitcoin_test = bitcoin_test.loc[bitcoin_test.index < '2020-04-25']

In [15]:
bitcoin_train = bitcoin_train.reset_index()
bitcoin_trainx = bitcoin_train[['Timestamp']]
bitcoin_trainx['Timestamp'] = pd.to_datetime(bitcoin_trainx['Timestamp'])
bitcoin_trainx['Timestamp']=bitcoin_trainx['Timestamp'].map(dt.datetime.toordinal)
bitcoin_trainx

<ipython-input-15-44349ceef573>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_trainx['Timestamp'] = pd.to_datetime(bitcoin_trainx['Timestamp'])
<ipython-input-15-44349ceef573>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_trainx['Timestamp']=bitcoin_trainx['Timestamp'].map(dt.datetime.toordinal)


,Timestamp
0,737046
1,737047
2,737048
3,737049
4,737050
...,...
466,737512
467,737513
468,737514
469,737515


In [16]:
bitcoin_trainy = bitcoin_train[['Close']]
bitcoin_trainy

,Close
0,3517.74
1,3715.25
2,3736.95
3,4017.46
4,3807.18
...,...
466,6109.16
467,6209.34
468,6017.46
469,6435.00


In [26]:
bitcoin_test= bitcoin_test.reset_index()

bitcoin_test

,index,Timestamp,Close
0,0,2020-04-02,6626.56
1,1,2020-04-03,6783.30
2,2,2020-04-04,6697.55
3,3,2020-04-05,6785.86
4,4,2020-04-06,6864.44
...,...,...,...
358,358,2021-03-26,52527.35
359,359,2021-03-27,54856.01
360,360,2021-03-28,56149.01
361,361,2021-03-29,55401.67


In [27]:
bitcoin_testx = bitcoin_test[['Timestamp']]
bitcoin_testx['Timestamp'] = pd.to_datetime(bitcoin_testx['Timestamp'])
bitcoin_testx['Timestamp']=bitcoin_testx['Timestamp'].map(dt.datetime.toordinal)
bitcoin_testx

<ipython-input-27-f3d0816fa189>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_testx['Timestamp'] = pd.to_datetime(bitcoin_testx['Timestamp'])
<ipython-input-27-f3d0816fa189>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_testx['Timestamp']=bitcoin_testx['Timestamp'].map(dt.datetime.toordinal)


,Timestamp
0,737517
1,737518
2,737519
3,737520
4,737521
...,...
358,737875
359,737876
360,737877
361,737878


In [28]:
bitcoin_testy = bitcoin_test[['Close']]
bitcoin_testy

,Close
0,6626.56
1,6783.30
2,6697.55
3,6785.86
4,6864.44
...,...
358,52527.35
359,54856.01
360,56149.01
361,55401.67


# Linear Regression

In [29]:
from sklearn.linear_model import LinearRegression

In [30]:
reg = LinearRegression()

In [31]:
reg = reg.fit(bitcoin_trainx,bitcoin_trainy)

In [32]:
bitcoin_predy = reg.predict(bitcoin_testx)

In [115]:
print("linear regression: ", mean_squared_log_error(bitcoin_test['Close'], bitcoin_predy))

linear regression:  0.4016838940072041


# Support Vector Machine

In [99]:
print(bitcoinx)

               Close  Prediction
Timestamp                       
2011-12-31      4.39        5.30
2012-01-01      4.84        5.70
2012-01-02      5.00        5.52
2012-01-03      5.32        6.16
2012-01-04      4.93        6.00
...              ...         ...
2021-03-26  52527.35         NaN
2021-03-27  54856.01         NaN
2021-03-28  56149.01         NaN
2021-03-29  55401.67         NaN
2021-03-30  57051.13         NaN

[3378 rows x 2 columns]


In [100]:
#A variable for predicting 'n' days out into the future
prediction_days = 30 
#n = 30 days

#Create another column (the target or dependent variable) shifted 'n' units up
bitcoinx['Prediction'] = bitcoinx[['Close']].shift(-prediction_days)

<ipython-input-100-8fe9d60001c3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoinx['Prediction'] = bitcoinx[['Close']].shift(-prediction_days)


In [101]:
bitcoinx.head()


,Close,Prediction
Timestamp,,
2011-12-31,4.39,5.30
2012-01-01,4.84,5.70
2012-01-02,5.00,5.52
2012-01-03,5.32,6.16
2012-01-04,4.93,6.00


In [102]:
bitcoinx.tail()

,Close,Prediction
Timestamp,,
2021-03-26,52527.35,NaN
2021-03-27,54856.01,NaN
2021-03-28,56149.01,NaN
2021-03-29,55401.67,NaN
2021-03-30,57051.13,NaN


In [103]:
#CREATE THE INDEPENDENT DATA SET (X)

# Convert the dataframe to a numpy array and drop the prediction column
X = np.array(bitcoinx.drop(['Prediction'],1))

#Remove the last 'n' rows where 'n' is the prediction_days
X= X[:len(bitcoin)-prediction_days]
print(X)

[[4.390000e+00]
 [4.840000e+00]
 [5.000000e+00]
 ...
 [4.719240e+04]
 [4.759500e+04]
 [4.433332e+04]]


In [104]:
#CREATE THE DEPENDENT DATA SET (y) 
# Convert the dataframe to a numpy array (All of the values including the NaN's) 
y = np.array(bitcoinx['Prediction'])  
# Get all of the y values except the last 'n' rows 
y = y[:-prediction_days] 
print(y)

[5.300000e+00 5.700000e+00 5.520000e+00 ... 5.614901e+04 5.540167e+04
 5.705113e+04]


In [105]:
# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [106]:
# Set prediction_days_array equal to the last 30 rows of the original data set from the price column
prediction_days_array = np.array(bitcoinx.drop(['Prediction'],1))[-prediction_days:]
print(prediction_days_array)

[[46481.83]
 [49138.32]
 [49196.4 ]
 [49203.53]
 [46737.32]
 [48439.71]
 [49428.12]
 [50528.08]
 [53785.49]
 [53620.44]
 [55735.57]
 [56708.33]
 [57032.02]
 [61199.06]
 [60528.22]
 [54150.19]
 [55475.97]
 [58802.57]
 [57808.56]
 [58299.31]
 [57778.8 ]
 [57637.17]
 [54638.56]
 [54279.85]
 [52050.  ]
 [52527.35]
 [54856.01]
 [56149.01]
 [55401.67]
 [57051.13]]


In [107]:
from sklearn.svm import SVR

In [108]:
# Create and train the Support Vector Machine 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.00001)#Create the model


In [109]:
svr_rbf.fit(x_train, y_train) #Train the model

SVR(C=1000.0, gamma=1e-05)

In [110]:
# Testing Model: Score returns the accuracy of the prediction. 
# The best possible score is 1.0
svr_rbf_confidence = svr_rbf.score(x_test, y_test)
print("svr_rbf accuracy: ", svr_rbf_confidence)

svr_rbf accuracy:  0.4920305104330943


In [111]:
# Print the predicted value
svm_prediction = svr_rbf.predict(x_test)
print(svm_prediction)

print()

#Print the actual values
print(y_test)

[8.31066440e+02 7.82671885e+02 2.40631874e+02 9.62224685e+03
 1.12240876e+01 1.20934754e+01 2.50099803e+02 1.12014675e+03
 1.06215085e+04 4.80060613e+02 6.17824568e+03 1.35802623e+04
 7.05770350e+01 1.22646693e+01 4.22725881e+02 6.09236353e+02
 8.01111588e+03 1.13990006e+04 2.58098874e+02 5.42027304e+02
 2.14317433e+02 7.59957017e+03 8.51404765e+03 1.02949728e+03
 1.50700621e+02 5.13386434e+03 5.20217992e+03 1.21910683e+01
 5.19387786e+03 8.11964116e+03 1.22064028e+01 2.43139372e+02
 9.32250319e+03 5.55313368e+02 6.83564413e+03 8.23775625e+03
 1.07953942e+01 7.47565752e+02 1.29162251e+04 1.07745132e+01
 4.98243535e+02 4.46632618e+02 4.24152399e+02 1.15921555e+04
 9.36298256e+01 4.70005336e+02 5.97640189e+02 8.84800821e+03
 7.62489868e+02 4.07507023e+02 1.08465404e+01 1.50344941e+02
 9.20697429e+03 1.07978992e+01 8.91314965e+03 6.90677616e+02
 5.14198884e+03 3.76912213e+02 1.22256912e+01 4.28822028e+02
 3.86371719e+02 8.34661215e+03 4.86983515e+02 5.16714122e+03
 2.70566166e+03 7.786652

In [112]:
# Print the model predictions for the next 'n=30' days
svm_prediction = svr_rbf.predict(prediction_days_array)
print(svm_prediction)

[11092.74148227 10744.28655286 10768.0769882  10772.01446454
 12301.40584026 10840.48789358 11082.76605118 12190.97277601
 10719.48139688 10719.48139688 12455.59873356 11717.18414838
 11027.73698365 10719.48139688 10719.48139688 10719.48139688
 11210.73249642 10719.48139688 10719.48499601 10719.48139688
 10719.48834323 10719.60594326 10719.48236095 10719.48139693
 10719.48155728 10719.48139688 10719.59227442 11607.68273362
 10987.47538586 10988.87182826]


In [114]:
#Print the actual price for the next 'n' days, n=prediction_days=30 
bitcoinx.tail(prediction_days)

,Close,Prediction
Timestamp,,
2021-03-01,46481.83,NaN
2021-03-02,49138.32,NaN
2021-03-03,49196.40,NaN
2021-03-04,49203.53,NaN
2021-03-05,46737.32,NaN
2021-03-06,48439.71,NaN
2021-03-07,49428.12,NaN
2021-03-08,50528.08,NaN
2021-03-09,53785.49,NaN
